In [ ]:
import numpy as np
import tensorflow as tf
import random, math
import matlab
import matlab.engine as me
import itertools
from myTensorUtil import *
from powMeth import *
from recmagsign import *

In [ ]:
## Activation Function
def phi_s(h):
    return h**2 if h > 0 else 0
phi = np.vectorize(phi_s)

In [ ]:
## moments
def gamma(j,sigma):
    estm = 0
    for i in range(10000):
        z = np.random.normal(0.0, 1.0)
        estm += phi_s(z*sigma)*(z**j)
    return estm/10000

In [ ]:
## Data and Params Generation
def generateWeights(d = 10, k = 5, s_gt = 0.15, kappa = 2 ):
    gauss_mat_u = np.random.normal(0.0, 1.0 , (d,k))
    gauss_mat_v = np.random.normal(0.0, 1.0 , (k,k))

    U, temp = np.linalg.qr(gauss_mat_u)
    V, temp = np.linalg.qr(gauss_mat_v)

    # U, V = np.linalg.qr(gauss_mat_u)
    diag = []
    v_gt = []
    v_choice = [1,-1]
    for iter in range(k):
        diag.append(1+1.*iter*(kappa-1)/(k-1))
        v_gt.append(random.choice(v_choice))

    Sigma = np.diag(diag)
    W_gt = np.dot(np.dot(U, Sigma), np.transpose(V))
    v_gt = np.asarray(v_gt)

#     s_gt = 1 
    # normW1 = []
    # for i in range(k):
    #     this_norm = np.linalg.norm(W_gt[:,i])
    #     normW1.append((this_norm,i))
    # normW1.sort()

    # for i in range(s_gt):
    #     this_ind = normW1[i][1] 
    #     W_gt[:,this_ind] = np.zeros(d)
    
    s_gt = 0.15
    for i in range(d):
        for j in range(k):
            this_norm = np.linalg.norm(W_gt[i,j])
            if this_norm <= s_gt:
                W_gt[i,j] = 0.0
                
    m = np.zeros((4,k))
    for i in range(k):
        m[0,i] = gamma(1,np.linalg.norm(W_gt[:,i]))
        m[1,i] = gamma(2,np.linalg.norm(W_gt[:,i])) - gamma(0,np.linalg.norm(W_gt[:,i]))
        m[2,i] = gamma(3,np.linalg.norm(W_gt[:,i])) - 3*gamma(1,np.linalg.norm(W_gt[:,i]))
        m[3,i] = gamma(4,np.linalg.norm(W_gt[:,i])) + 3*gamma(0,np.linalg.norm(W_gt[:,i])) - 6*gamma(2,np.linalg.norm(W_gt[:,i]))
    return [W_gt, v_gt, m]

In [ ]:
def structDiff(w_gt, w_res, delta = 1e-4):
    '''
    Returns :
    False Alarm --  GT - 0 Res- Non 0
    Mis Detection -- GT - Non 0 Res - 0
    True Positives -- GT - 0 Res - 0
    True Negatives -- GT - Non 0 Res - Non 0
    '''
    
    w_gt = w_gt.flatten()
    w_res = w_res.flatten()
    
    num_w = len(w_gt)
    assert(num_w == len(w_res)), 'Dimension not Equal\n'
    
    false_alarm, mis_det, tr_positive, tr_negative = 0,0,0,0
    
    for w in range(num_w):
        if abs(w_gt[w]) < delta and abs(w_res[w]) > delta :
            false_alarm += 1
        elif abs(w_gt[w]) > delta and abs(w_res[w]) < delta :
            mis_det += 1
        elif abs(w_gt[w]) < delta and abs(w_res[w]) < delta :
            tr_positive += 1
        elif abs(w_gt[w]) > delta and abs(w_res[w]) > delta :
            tr_negative += 1
        
    return [false_alarm, mis_det, tr_positive, tr_negative]

In [ ]:
def generateData(W_gt, v_gt,n = 2000, test_n = 1000, d = 10):

    train_x = []
    train_y = []
    for iter in range(n):
        train_x.append(np.random.normal(0.0,1.,d))
        train_y.append(np.dot(phi(np.dot(train_x[iter], W_gt)),v_gt))
    train_x = np.asarray(train_x)
    train_y = np.transpose(np.asarray(train_y))

    test_x = []
    test_y = []
    for iter in range(test_n):
        test_x.append(np.random.normal(0.0,1.,d))
        test_y.append(np.dot(phi(np.dot(test_x[iter], W_gt)),v_gt))
    test_x = np.asarray(test_x)
    test_y = np.transpose(np.asarray(test_y))

    return [train_x, train_y, test_x, test_y]

In [ ]:
### Implementation Verification

# m[2,3]/np.linalg.norm(W_gt[:,3])**2

# M =  np.asarray(getM2(train_x, train_y))

# check = np.zeros((d,d))
# # check = np.zeros(d)
# for i in range(k):
#     check += v_gt[i]*m[1,i]*np.outer(W_gt[:,i], W_gt[:,i])
# #     check += v_gt[i]*m[0,i]*W_gt[:,i]
# print check
# print np.linalg.norm(M-check)/np.linalg.norm(check)
# print np.linalg.norm(M-check)/np.linalg.norm(M)

In [ ]:
def tensorInit(X, y, W_gt, m, k, eng):
    divInd = int(len(X)/3)
    
    # Partition
    X1 = X[:divInd]
    y1 = y[:divInd]

    X2 = X[divInd:2*divInd]
    y2 = y[divInd:2*divInd]
    
    X3 = X[2*divInd:]
    y3 = y[2*divInd:]
    
    ## P2 
    ## Estimating P2 as M2
    
    ## Power Method
    V = powMeth(k, X1, y1)
    
    ## R3
    R3 = multLnr2(getM3(X2, y2), V)
    
    #### KCL
    R = matlab.double(R3.flatten().tolist())
#     eng = me.start_matlab()
    U = eng.notf_frompy(R, 100, k)
#     eng.quit()
    U = np.asarray(U)
    #U = R3[0]

    ## RecMagSign
    return recmagsgn(V, U, X3, y3, m, W_gt) 

In [ ]:
def init_weights(shape):
    """ Weight initialization """
    weights = tf.random_normal(shape, stddev=1.0)
    return tf.Variable(weights)

def forwardprop(X, w_1, w_2):
    """
    Forward-propagation.
    """
    h    = tf.square(tf.nn.relu((tf.matmul(X, w_1))))
    yhat = tf.matmul(h, w_2)  # The \varphi function
    return yhat

In [ ]:
def train(train_x, train_y, test_x, test_y, tensorWeights, v_gt, s=0.15, T = 20, batch_size = 10, lr = 1e-3,epsilon = 1e-4):

    d = np.shape(train_x)[1]
    n = np.shape(train_x)[0]
    test_n = np.shape(test_x)[0]
    k = np.shape(v_gt)[0]

    X = tf.placeholder("float64", shape=[None, d])
    y = tf.placeholder("float64", shape=[None, 1])

    # Weight initializations
    # w_1 = init_weights((d, k))
    # w_1 = tf.Variable(tf.cast(w_1, tf.float64))
    w_1 = tf.Variable(np.transpose(tensorWeights[0]))

    # w_2 = []
    # v_choice = [1,-1]
    # for iter in range(k):
    #     w_2.append(random.choice(v_choice))
    # w_2 = tf.cast(tf.Variable(np.transpose(np.asarray([w_2])), trainable=False), tf.float32)
    # w_2 = init_weights((k, 1))

    # w_2 = tf.Variable(tensorWeights[1], trainable=False)

    w_2 = tf.Variable( np.transpose([v_gt]), trainable=False)
    w_2 = tf.cast(w_2, tf.float64)

    # Forward propagation
    yhat  = forwardprop(X, w_1, w_2)

    # Backward propagation
    cost = tf.losses.mean_squared_error(y, yhat)
    updates = tf.train.GradientDescentOptimizer(lr).minimize(cost)

    # Run SGD
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    sess = tf.Session(config=config)
    init = tf.global_variables_initializer()
    sess.run(init)
    epsilon = 1e-4
    train_loss = []
    test_loss = []
    T = 20

    for epoch in range(T):

        ## Shuffling
        permList = np.random.permutation(range(n))
        train_x = train_x[permList]
        train_y = train_y[permList]

         ## Iterative Hard Thresholding
        ## after every epoch

        ## Node Removal
    #     cur_w1 =  w_1.eval(session=sess)   
    #     normW1 = []
    #     for i in range(k):
    #         this_norm = np.linalg.norm(cur_w1[:,i])
    #         normW1.append((this_norm,i))
    #     normW1.sort()

    #     for i in range(s):
    #         this_ind = normW1[i][1] 
    #         cur_w1[:,this_ind] = np.zeros(d)
    #     assign_op = tf.assign(w_1, cur_w1)
    #     sess.run(assign_op)

        ## Connection Removal
    #     cur_w1 =  w_1.eval(session=sess)   
    #     for i in range(d):
    #         for j in range(k):
    #             this_norm = np.linalg.norm(cur_w1[i,j])
    #             if this_norm <= s:
    #                 cur_w1[i,j] = 0.0
    #     assign_op = tf.assign(w_1, cur_w1)
    #     sess.run(assign_op)

        ## Connection Removal in a node
    #     cur_w1 =  w_1.eval(session=sess)   
    #     normW1 = []
    #     for i in range(k):
    #         this_norm = np.linalg.norm(cur_w1[:,i])
    #         normW1.append((this_norm,i))
    #     normW1.sort()
    #     check = 0
    #     for i in range(s):
    #         this_ind = normW1[i][1] 
    #         for j in range(d):
    #             this_norm = np.linalg.norm(cur_w1[j,this_ind])
    #             if this_norm <= connection_thresh:
    #                 cur_w1[j,this_ind] = 0.0
    #                 check += 1
    #     assign_op = tf.assign(w_1, cur_w1)
    #     sess.run(assign_op)

        i = 0    
        for iter in range(int(n/batch_size)):
            sess.run(updates, feed_dict={X: train_x[i: i + batch_size], y: train_y[i: i + batch_size].reshape(batch_size,1)})
            i = (i + batch_size)%n
    #     train_accuracy = np.mean((train_y - sess.run(yhat, feed_dict={X:train_x})) <= epsilon)
        loss = sess.run(cost, feed_dict={X:train_x, y:train_y.reshape(n,1)})
        testLoss = sess.run(cost, feed_dict={X:test_x, y:test_y.reshape(test_n,1)})
        
        print "Epoch = ", epoch+1,"training loss: ", loss ," test loss: ",testLoss 
        
        train_loss.append(loss)
        test_loss.append(testLoss)
    return sess.run(w_1), train_loss, test_loss

In [ ]:
def recovery(W_gt, v_gt, w_res, v_res ):
    
    k = np.shape(v_gt)[0]
    permList = list(itertools.permutations(range(k)))

    w_p = np.transpose(w_res)
    
    v_p = v_res
    
    min_v = 100
    for perm in permList:
        w_pi = w_p[list(perm)]
        v_pi = v_p[list(perm)]

        w_gt = np.transpose(W_gt)

        if sum(v_gt == v_pi) == k:
            max_diff = 0
            for i in range(k):
                diff = np.linalg.norm(w_pi[i]-w_gt[i])/np.linalg.norm(w_gt)
                if diff > max_diff:
                    max_diff = diff
            if max_diff < min_v:
                min_v = max_diff
        else:
            continue
    return min_v

In [ ]:
n = 2000
test_n = 1000
d = 10
k = 5
thresh_gt = 0.15
thresh_train = 0.15
batch_size = 20
lr = 1e-3

w_gt, v_gt, m =  generateWeights(d, k, thresh_gt)
train_x, train_y, test_x, test_y = generateData(w_gt, v_gt, n, test_n, d)

eng = me.start_matlab()
tensorWeights = tensorInit(train_x, train_y, w_gt, m ,k, eng)
eng.quit()

In [ ]:
w_res, train_loss, test_loss = train(train_x, train_y, test_x, test_y, tensorWeights,v_gt )

In [ ]:
recoveryVal = recovery(w_gt, v_gt, w_res, v_gt)
print recoveryVal

recoveryStructure = structDiff(w_gt, w_res, 1e-2)
print recoveryStructure

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
NT = 10

y_tl0 = y_tl0[:NT]
acc0 = acc0[:NT]

y_tl1 = y_tl1[:NT]
acc1 = acc1[:NT]

y_tl2 = y_tl2[:NT]
acc2 = acc2[:NT]

y_tl3 = y_tl3[:NT]
acc3 = acc3[:NT]

# y_tl4 = y_tl4[:NT]
# acc4 = acc4[:NT]
x_e = x_e[:NT]

In [ ]:
fig, ax1 = plt.subplots()
ax1.plot(a, b, 'blue', label="Ground Truth Thresh= 0.20")
ax1.set_xlabel('threshold', color="white")
# Make the y-axis label, ticks and tick labels match the line color.
ax1.set_ylabel('recovery', color='white')
ax1.tick_params('y', colors='white')
ax1.tick_params('x', colors='white')
l1 = ax1.legend(bbox_to_anchor=(1.1 ,0.5), loc=2, borderaxespad=0.)
plt.show()

In [ ]:
fig, ax1 = plt.subplots()
ax1.plot(x_e, y_tl0, 'blue', label="Ground Truth Thresh= 0.15")
# ax1.plot(x_e, y_tl, 'red', label="random")
ax1.plot(x_e, y_tl1, 'red', label="thresh = 0.10")
ax1.plot(x_e, y_tl2, 'green', label="thresh=0.15")
ax1.plot(x_e, y_tl3, 'black', label="thresh=0.20")
ax1.plot(x_e, y_tl4, 'magenta', label="thresh=0.25")
ax1.set_xlabel('epochs', color="white")
# Make the y-axis label, ticks and tick labels match the line color.
ax1.set_ylabel('training loss', color='white')
ax1.tick_params('y', colors='white')
ax1.tick_params('x', colors='white')
l1 = ax1.legend(bbox_to_anchor=(1.1 ,0.5), loc=2, borderaxespad=0.)
# ax1.set_ylim([0.20,0.30])


# ax2 = ax1.twinx()
fig, ax2 = plt.subplots()
ax2.plot(x_e, acc0, 'blue', label="Ground Truth Thresh= 0.15")
# ax2.plot(x_e, acc, 'red', label="random_acc")
ax2.plot(x_e, acc1, 'red',label="thresh=0.10")
ax2.plot(x_e, acc2, 'green',label="thresh=0.15")
ax2.plot(x_e, acc3, 'black',label="thresh=0.20")
ax2.plot(x_e, acc4, 'magenta',label="thresh=0.25")
ax2.set_ylabel('test error', color='white')
ax2.tick_params('y', colors='white')
ax2.tick_params('x', colors='white')
ax2.set_xlabel('epochs', color="white")
l2 = ax2.legend(bbox_to_anchor=(1.1, 1), loc=2, borderaxespad=0.)
# ax2.set_ylim([0.20,0.30])
# 
# fig.tight_layout()
plt.show()